In [ ]:
from unified_planning.shortcuts import *
from unified_planning.plans import *
from importlib import reload
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
import unifiedplanning_blocksworld as bw
import prompts
import planbench as pb
import policy_model as policy
import llm_utils
import data_loader
import pandas as pd
import regex as re

In [ ]:
model='google/gemma-2-9b-it'
temp=0

In [ ]:
reload(policy)
reload(prompts)
reload(bw)
reload(pb)
reload(llm_utils)
reload(data_loader)

#### Vanilla prompting ####

In [ ]:
llm_utils.query_llm('help')

In [ ]:
problem = bw.BlocksworldProblem()

In [ ]:
pb.parse_planbench_initial_condition(problem, prompts.nextaction_problem_init)
pb.parse_planbench_goal_state(problem, prompts.nextaction_problem_goal)
#print(problem.initial_values)
#initial vals ko vaapais readable mein translate karna hoga

In [ ]:

pi = policy.PolicyModel()
    # def next_action_one_shot(self, 
    #                          problem_init, problem_goal,problem_action_hist,  # the state S
    #                          example_init, example_goal,example_action_hist,example_next_action)
plan=[]
#action_tuple = pi.next_action_one_shot(prompts.nextaction_problem_init, prompts.nextaction_problem_goal,
#                                       plan,
#                                       prompts.nextaction_example_init, prompts.nextaction_example_goal,
#                                       prompts.nextaction_example_action_hist,prompts.nextaction_example_nextaction)


In [ ]:
action_tuple=('unstack','blue','orange')
#print(action_tuple)
next_action=problem.create_action(action=action_tuple[0],blocks=action_tuple[1:])
#print(next_action)

In [ ]:
simulation=problem.create_seq_simulation()
current_state=simulation.get_initial_state()
new_state=problem.validate_action(current_state,next_action,simulation)
#if(new_state):
    #print(new_state)
#else:
    #print('Invalid action proposed by policy')

In [ ]:
reload(data_loader)
df=data_loader.load_data("../data/train.parquet")
df.head()


In [ ]:
#df.size

In [ ]:
def evaluate(attempts,struct,diff_len,results,actions_to_goal,gplan_lens,valid_action_count):
    metrics=pd.DataFrame({
        'Parsable':struct,
        'Attempts':attempts,
        'Terminated':results,
        'Valid Actions':valid_action_count,
        'Distance from goal':actions_to_goal,
        'Diff in plan len':diff_len,
        'GOLD Plan len':gplan_lens
    })
    correct_plans=sum(results)
    c_rate=correct_plans/len(results)
    c_rate=format(c_rate,".2f")
    return metrics,c_rate

In [ ]:
#vanilla method
from collections import defaultdict
import math
def main_fullSol(df,model,temp):
    results=[False]*len(df)
    well_struct=[False]*len(df)
    num_tries=[0]*len(df)
    diff_planlen=[-math.inf]*len(df)
    actions_to_goal=[-math.inf]*len(df)
    gplan_lens=[-math.inf]*len(df)
    valid_action_count=[0]*len(df)
    for i in range(0,len(df)):
    #for i in range(0,1):
        
        print(f'\n\n--------------------Entering main loop PROBLEM #{i}--------------------')
        
        #declarations: Blocksworld Problem object
        problem = bw.BlocksworldProblem()
        model_plan=None

        #create prompt using PLANBENCH dataset
        #prompt=prompts.make_prompt(df.iloc[i]['query'])
        #print(f'\n\nMake prompt:{prompt} \nEND Make prompt')

        #create prompt using OUR dataset
        prompt,init,goal=prompts.make_prompt(df.iloc[i]['init'],df.iloc[i]['goal'],df.iloc[i]['demo_init'],df.iloc[i]['demo_goal'],df.iloc[i]['demo_plan'])

        #parse PLANBENCH dataset into Blocksworld object
        #pbstring=prompts.get_pbstring(df.iloc[i]['query'])
        #init,goal=pb.parse_problem(pbstring)
        #pb.parse_planbench_initial_condition(problem, init)
        #pb.parse_planbench_goal_state(problem, goal)
        #print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
        #print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')
        
        #changed generation of Blocksworld object for OUR dataset
        pb.parse_planbench_initial_condition(problem, init)
        pb.parse_planbench_goal_state(problem, goal)
        print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
        print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')

        #activate solve mode
        pi = policy.PolicyModel()
        #query the llm, parse the actions, return tuples
        tags="Answer within the [PLAN] [PLAN END] tags."
        cot="Step by step reason about the actions you choose to take."
        response=pi.Vanilla_fullSol_one_shot(prompt+"\n"+cot+"\n"+tags,model=model,temp=temp)
        actions=response[0]
        tries=response[1]

        num_tries[i]=tries

        if not actions:
            well_struct[i]=False
            print(f'\n\n LLM exceeded 3 tries')
            results[i]=False
            continue
        well_struct[i]=True
        plan_len=len(actions)
        
        #generate plan from proposed actions
        #actions=[('unstack', 'brown', 'teal'), ('put-down', 'brown'), ('unstack', 'teal', 'violet'), ('put-down', 'teal'), ('unstack', 'violet', 'teal'), ('put-down', 'violet')] 
        #actions=[('unstack', 'green', 'pink'), ('put-down', 'green'), ('unstack', 'pink', 'red'), ('put-down', 'pink'),('pick_up', 'green'), ('stack', 'green' , 'red')] 
        #plan_len=len(actions)

        print(f'\n\nProposed actions by LLM : {actions} \n  END')
        model_plan=problem.create_plan_from_tuples(actions)
        if not model_plan:
            results[i]=False
            print('!!!!!!!!!!!Action statement NOT FROM LIST OF ALLOWED actions!!!!!!!!!!!!')
            #continue

        if model_plan:
            print(f'LLM PLAN: {model_plan}')

        #validate and apply    
        simulation=problem.create_seq_simulation()
        va_counter=0
        valid_state,flag,va_counter=problem.check_and_apply(simulation,model_plan)
        valid_action_count[i]=va_counter
        if not flag:
            results[i]=False
            #continue
            

        #compute actions to goal from last valid state
        # print(f'UBS OBJECT FOR LAST VALID STATE: {valid_state,sim}')
        a2g=problem.actions_to_goal(valid_state)
        actions_to_goal[i]=a2g

        
        #get gold-plan on problem
        gold_plan=problem.generate_plan()
        gplan_actions=str(gold_plan.plan).strip().split('\n')
        gplan_actions=gplan_actions[1:]
        gplan_len=len(gplan_actions)
        gplan_lens[i]=gplan_len
        print(f'\n\n$$$Proposed actions by GOLD PLAN $$$ : {gplan_actions} \n END')


        #termination-check 
        if problem.terminate(valid_state):
            # print(f'Check and Apply return STATE:{valid_state}')
            # print(f'v/s')
            # print(f'GOAL STATE: {problem.goals}')
            # print('Setting result as true')
            results[i]=True
            diff_planlen[i]=plan_len-gplan_len
            continue
        
        results[i]=False

        #compute metrics

    metrics,c_rate=evaluate(num_tries,well_struct,diff_planlen,results,actions_to_goal,gplan_lens,valid_action_count)
    return metrics,c_rate

In [ ]:
#load data for PLANBENCH dataset
#df=data_loader.load_data("../data/train.parquet")
#df.head()
#results=main_fullSol(df)

In [ ]:
#print(f"\n\n Model solution's verdict: {results}")

In [ ]:
#df.head

In [ ]:
#load data for OUR dataset
import pandas as pd
n=3
df=pd.read_csv(f'../data/{n}_blocks/Oneshot_val_{n}_blocks')
df.drop(columns='Unnamed: 0',inplace=True)
df

In [ ]:
# df=df[1:2]
# df

In [ ]:
df.shape

In [ ]:
model='google/gemma-2-9b-it'
temp=0

In [ ]:
response=main_fullSol(df,model=model,temp=temp)

In [ ]:
metrics_df=response[0]
c_rate=response[1]

In [ ]:
#metrics_df=pd.read_csv('../data/results/Vanilla_FullPlanGeneration_Val_3_blocks')

In [ ]:
print(f'----------------{model}, w COT, w demo help, w plan-end tags----------')
metrics_df

In [ ]:
print(f'Correctness rate : {c_rate}')
print(f"Goal State Reached: {sum(metrics_df['Distance from goal']==0)}")

In [ ]:
#metrics_df.to_csv(f'../data/results/Vanilla_FullPlanGeneration_Val_3_blocks')

In [ ]:
problem=bw.BlocksworldProblem()

In [ ]:
BLOCK='green'
blocks={'green':'green'}
block=blocks.get(BLOCK)
block

In [ ]:
len(df)

In [ ]:
#qwen
Qwen/Qwen2-7B-Instruct
Qwen/Qwen2.5-7B-Instruct

#llama
meta-llama/Meta-Llama-3-8B-Instruct
meta-llama/Meta-Llama-3.1-8B-Instruct
meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo

#mistral
mistralai/Mistral-7B-Instruct-v0.1
mistralai/Mistral-7B-Instruct-v0.2
mistralai/Mistral-7B-Instruct-v0.3

#phi-4
microsoft/Phi-4-multimodal-instruct

#gemma
google/gemma-1.1-7b-it
google/gemma-3-4b-it
google/gemma-2-9b-it

#### COT: Trials ####

In [ ]:
reasons=[]

In [ ]:
reason='''I faced the stubborn tower: Brown on Violet, Violet on Teal, Teal on table. The goal? Brown on table, Violet on Brown, Teal on Violet. My fingers flexed.

"Brown's clear—I'll move it first." I unstuck Brown from Violet, cradling it. Wait. The goal wants Brown on table... so far so good. I set Brown down with a clunk.

Now Violet sat exposed atop Teal. "Need Violet on Brown... but first, is Violet clear?" I checked—yes, no blocks above. I unstacked Violet, then hesitated. "Brown's ready—but is it still clear?" I nudged it. Good, untouched. I stacked Violet onto Brown.

Teal remained alone on the table. "Now for the finale..." I reached, then froze. "Is Violet clear for Teal?" I peered—Violet's surface bare. Safe. I lifted Teal, stacked it atop Violet.

The new tower stood: Brown (table) → Violet → Teal.

Not Just Moving Blocks:
Each action trailed by silent questions—"Can I? Should I?"—until the blocks aligned not by speed, but by relentless verification.

Moral: Precision isn't in the moves; it's in the moments between.

'''

In [ ]:
reasons.append(reason)

In [ ]:
i=1
for r in reasons:
    print(f'{i}. {r}')
    i+=1

In [ ]:
prompt=('''I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block.
        I have the following restrictions on my actions:
        I can only pick up or unstack one block at a time
        I can only pick up or unstack a block if my hand is empty
        I can only pick up a block if the block is on the table and the block is clear
        A block is clear if the block has no other blocks on top of it and if the block is not picked up
        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block
        I can only unstack a block from on top of another block if the block I am unstacking is clear
        Once I pick up or unstack a block, I am holding the block
        I can only put down a block that I am holding
        I can only stack a block on top of another block if I am holding the block being stacked
        I can only stack a block on top of another block if the block onto which I am stacking the block is clear
        Once I put down or stack a block, my hand becomes empty
        Once you stack a block on top of a second block, the second block is no longer clear

[STATEMENT]
As initial conditions I have that, the green block is clear,  the hand is empty, the red block is on the table, the pink block is on top of the red block, the green block is on top of the pink block.
My goal is to have that  the green block is on the table, the red block is on top of the green block, the pink block is on top of the red block.

Wait, let's think step by step.
[REASONING]
'''

f"\n \n{reason}\n\n [REASONING END]"
'''
My plan is as follows:

[PLAN]
unstack the green block from on top of the pink block
put down the green block
unstack the pink block from on top of the red block
put down the pink block
pick up the red block
stack the red block on top of the green block
pick up the pink block
stack the pink block on top of the red block
[PLAN END]

[STATEMENT]
As initial conditions I have that, the brown block is clear,  the hand is empty, the violet block is on the table, the teal block is on top of the violet block, the brown block is on top of the teal block.
My goal is to have that  the violet block is on the table, the teal block is on the table, the brown block is on the table.

Wait, let's think step by step.
[REASONING]

Reason about the steps you take. Finally, answer within the [PLAN] [PLAN END] tags.''')


In [ ]:
results=[]
for i in range(0,5):
    prompt,init,goal=prompts.cot_make_prompt(df.iloc[i]['init'],df.iloc[i]['goal'],df.iloc[i]['demo_init'],df.iloc[i]['demo_goal'],df.iloc[i]['demo_plan'],reasons[i])
    print(f'----------------------------------PROMPT {i}------------------------:\n\n{prompt}')
    r=llm_utils.query_llm(prompt,model=model,temperature=temp)
    results.append(r)

In [ ]:
for r in results:
    print(r)

In [ ]:
import pandas as pd
from io import StringIO

In [ ]:
# Clean up leading/trailing spaces
markdown_table = markdown_table.strip()

# Read it as if it's a CSV, but with "|" separator
df = pd.read_csv(StringIO(markdown_table), sep="|", engine="python")

# Drop the weird first and last empty columns (caused by the leading/trailing '|')
df = df.drop(columns=["Unnamed: 0", "Unnamed: 9"])

# Optionally strip spaces from column names
df.columns = df.columns.str.strip()

print(df)

In [ ]:
df.drop(columns=['Index'],inplace=True)

In [ ]:
df.head(11)

In [ ]:
df.to_csv('')

In [ ]:
df=df.rename(columns=
          {':-----------':'Parsable',
           '-----------:':'Attempts',
           ':-------------':'Terminated',
           '----------------:':'Valid Actions',
           '---------------------:':'Distance from goal',
           '-------------------:':'Diff in plan len',
           '----------------:.1':'GOLD Plan len',})

#### Local LLM set-up ####

Strategise at each step before you choose an action, then form a final plan.
Step by step reason about the actions you choose to take.

In [ ]:
prompt1='''I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block.
        I have the following restrictions on my actions:
        I can only pick up or unstack one block at a time
        I can only pick up or unstack a block if my hand is empty
        I can only pick up a block if the block is on the table and the block is clear
        A block is clear if the block has no other blocks on top of it and if the block is not picked up
        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block
        I can only unstack a block from on top of another block if the block I am unstacking is clear
        Once I pick up or unstack a block, I am holding the block
        I can only put down a block that I am holding
        I can only stack a block on top of another block if I am holding the block being stacked
        I can only stack a block on top of another block if the block onto which I am stacking the block is clear
        Once I put down or stack a block, my hand becomes empty
        Once you stack a block on top of a second block, the second block is no longer clear

[STATEMENT]
As initial conditions I have that, the green block is clear,  the hand is empty, the pink block is on the table, the red block is on top of the pink block, the green block is on top of the red block.
My goal is to have that  the green block is on the table, the pink block is on top of the green block, the red block is on the table.

My plan is as follows:

[PLAN]
unstack the green block from on top of the red block
put down the green block
unstack the red block from on top of the pink block
put down the red block
pick up the pink block
stack the pink block on top of the green block
[PLAN END]

[STATEMENT]
As initial conditions I have that, the red block is clear, the green block is clear,  the hand is empty, the pink block is on the table, the green block is on top of the pink block, the red block is on the table.
My goal is to have that  the pink block is on the table, the red block is on top of the pink block, the green block is on the table.

[PLAN]
Answer within the [PLAN] [PLAN END] tags.
'''

In [ ]:
import torch
#torch.cuda.empty_cache()
model_name='google/gemma-2-9b-it'
temp=0

In [ ]:
#model,tokenizer=llm_utils.get_model_tokenizer(model_name)

In [ ]:
response=llm_utils.query_local_model(model=model,tokenizer=tokenizer,model_name=model_name,temperature=0,prompt=prompt1)

In [ ]:
r=llm_utils.query_llm(model=model_name,temperature=temp,prompt=prompt)

In [ ]:
print(r)

In [ ]:
r=llm_utils.query_llm(prompt=prompt1+"Let's DeepThink this.",model='deepseek-ai/DeepSeek-R1-Distill-Llama-70B',temperature=0,max_tokens=1200)

In [ ]:
print(r)

In [ ]:
demo_prompt=prompt1.split('[PLAN]')[0]
print(demo_prompt)

In [ ]:
train_3=pd.read_csv(f'../data/3_blocks/SFT_train_3_blocks_fullPlan').drop(columns='Unnamed: 0')

In [ ]:
def get_deep_thoughts(df,model,temp):
    thoughts=[]
    for i in range(len(df)):
        print(f'\n\n--------------------Entering main loop PROBLEM #{i}--------------------')
        #prompt,init,goal=prompts.make_prompt(df.iloc[i]['init'],df.iloc[i]['goal'],df.iloc[i]['demo_init'],df.iloc[i]['demo_goal'],df.iloc[i]['demo_plan'])
        prompt=df['prompt'][i]
        demo_prompt=prompt.split('[PLAN]')[0].strip()
        demo_prompt=demo_prompt+'[PLAN]'
        r=llm_utils.query_llm(prompt=demo_prompt+"Let's DeepThink this.",model=model,temperature=temp,max_tokens=1200)
        thought=r.split('<think>')[1].split('</think>')[0].strip()
        thoughts.append(thought)
    return thoughts


In [ ]:
model='deepseek-ai/DeepSeek-R1-Distill-Llama-70B'
temp=0

In [ ]:
thoughts=get_deep_thoughts(train_3,model,temp)

In [ ]:
thoughts_df=pd.DataFrame(thoughts,columns=['thought'])

In [ ]:
thoughts_df.to_csv('../data/3_blocks/thoughts.csv')

In [ ]:
Parsable',
      3            '-----------:':'Attempts',
      4            ':-------------':'Terminated',
      5            '----------------:':'Valid Actions',
      6            '---------------------:':'Distance from goal',
      7            '-------------------:':'Diff in plan len',
      8            '----------------:.1':'GOLD Plan len',})

In [ ]:
import pandas as pd
base_dir="/srv/chawak/planning-with-llms/results/rl/training/"
inference_dir="inference/inference_24_06/train/tags_checkpoint_120/metrics.csv"
inf_csv=base_dir+inference_dir
df=pd.read_csv(f'{inf_csv}')
df.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
df

In [ ]:
import sys
sys.path.append("/srv/chawak/planning-with-llms/src/")

In [ ]:
from shared import llm_utils